# Day 19 
## Part 1
Far too tempting. Let's compile this to Python.

In [1]:
import parse

TAB = "    "

def compile_result(result):
    if result in "AR":
        return f'"{result}"'
    return f"{result}(x)"

def compile_workflow(workflow):
    lines = []
    r = parse.parse("{label}{{{rules}}}", workflow)
    if r["label"] != "in":
        label = r["label"]
    else:
        label = "start"
    lines.append(f"def {label}(x):")
    rules = r["rules"].split(",")
    for rule in rules[:-1]:
        cond, result = rule.split(":")
        lines.append(TAB + f'if x["{cond[0]}"] {cond[1]} {cond[2:]}:')
        lines.append(TAB * 2 + f"return {compile_result(result)}")
    lines.append(TAB + f"return {compile_result(rules[-1])}")
    return "\n".join(lines)

print(compile_workflow("px{a<2006:qkq,m>2090:A,rfg}"))

def px(x):
    if x["a"] < 2006:
        return qkq(x)
    if x["m"] > 2090:
        return "A"
    return rfg(x)


In [2]:
import re

def compile_part(part):
    return re.sub(r"(.)=", r'"\1":', part)

compile_part("{x=787,m=2655,a=1222,s=2876}")

'{"x":787,"m":2655,"a":1222,"s":2876}'

In [3]:
def compile(input):
    workflows, parts = input.split("\n\n")
    code = "\n\n".join([
        compile_workflow(workflow)
        for workflow in workflows.splitlines()
    ])
    data = (
        "parts = [\n" 
        + ",\n".join([
            TAB + compile_part(part)
            for part in parts.splitlines()
        ])
        + "\n]\n"
    )
    return code + "\n\n" + data

In [4]:
test_input = """px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}
qkq{x<1416:A,crn}
crn{x>2662:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,m<1801:hdj,R}
gd{a>3333:R,R}
hdj{m>838:A,pv}

{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}"""

print(compile(test_input))

def px(x):
    if x["a"] < 2006:
        return qkq(x)
    if x["m"] > 2090:
        return "A"
    return rfg(x)

def pv(x):
    if x["a"] > 1716:
        return "R"
    return "A"

def lnx(x):
    if x["m"] > 1548:
        return "A"
    return "A"

def rfg(x):
    if x["s"] < 537:
        return gd(x)
    if x["x"] > 2440:
        return "R"
    return "A"

def qs(x):
    if x["s"] > 3448:
        return "A"
    return lnx(x)

def qkq(x):
    if x["x"] < 1416:
        return "A"
    return crn(x)

def crn(x):
    if x["x"] > 2662:
        return "A"
    return "R"

def start(x):
    if x["s"] < 1351:
        return px(x)
    return qqz(x)

def qqz(x):
    if x["s"] > 2770:
        return qs(x)
    if x["m"] < 1801:
        return hdj(x)
    return "R"

def gd(x):
    if x["a"] > 3333:
        return "R"
    return "R"

def hdj(x):
    if x["m"] > 838:
        return "A"
    return pv(x)

parts = [
    {"x":787,"m":2655,"a":1222,"s":2876},
    {"x":1679,"m":44,"a":2067,"s":496},
    

In [5]:
exec(compile(test_input))

In [6]:
[sum(part.values()) for part in parts if start(part) == "A"]

[7540, 4623, 6951]

In [7]:
def part_1(input):
    # this is so bad
    exec(compile(input), globals())
    return sum(sum(part.values()) for part in parts if start(part) == "A")

assert part_1(test_input) == 19114

In [8]:
input = open("input").read()

part_1(input)

330820

## Part 2
Well that approach was fun but isn't going to be useful for this. Start again. 

In [9]:
from collections import defaultdict, namedtuple

def parse_data(input):
    workflows = defaultdict(list)
    lines = input.strip().split("\n\n")[0].splitlines()
    for line in lines:
        r = parse.parse("{label}{{{rules}}}", line)
        rules = r["rules"].split(",")
        for rule in rules[:-1]:
            comp, res = rule.split(":")
            workflows[r["label"]].append((
                comp[0], 
                comp[1], 
                int(comp[2:]),
                res
            ))
        workflows[r["label"]].append((rules[-1],))
    return workflows

parse_data(test_input)

defaultdict(list,
            {'px': [('a', '<', 2006, 'qkq'), ('m', '>', 2090, 'A'), ('rfg',)],
             'pv': [('a', '>', 1716, 'R'), ('A',)],
             'lnx': [('m', '>', 1548, 'A'), ('A',)],
             'rfg': [('s', '<', 537, 'gd'), ('x', '>', 2440, 'R'), ('A',)],
             'qs': [('s', '>', 3448, 'A'), ('lnx',)],
             'qkq': [('x', '<', 1416, 'A'), ('crn',)],
             'crn': [('x', '>', 2662, 'A'), ('R',)],
             'in': [('s', '<', 1351, 'px'), ('qqz',)],
             'qqz': [('s', '>', 2770, 'qs'), ('m', '<', 1801, 'hdj'), ('R',)],
             'gd': [('a', '>', 3333, 'R'), ('R',)],
             'hdj': [('m', '>', 838, 'A'), ('pv',)]})

Use intervals with minimum and maximum values to track the possible values through the control flow. When hitting an "A" add the product of the size of the intervals, i.e. the number of combinations of values that could hit that "A".

In [10]:
from pyrsistent import pmap
import math

Interval = namedtuple("Interval", "min max")

def part_2(input):
    workflows = parse_data(input)

    def sum_acceptable(workflow, intervals):
        if workflow == "R" or any(
            intervals[i].min > intervals[i].max 
            for i in intervals
        ):
            return 0
        if workflow == "A":
            return math.prod(
                max(intervals[i].max - intervals[i].min + 1, 0)
                for i in intervals
            )
        total = 0
        for w in workflows[workflow]:
            match w:
                case ("R",):
                    pass
                case ("A",):
                    total += math.prod(
                        max(intervals[i].max - intervals[i].min + 1, 0)
                        for i in intervals
                    )
                case (wf,):
                    total += sum_acceptable(wf, intervals)
                case (rating, op, n, res):
                    i = intervals[rating]
                    if op == "<":
                        new_intervals = intervals.set(rating, Interval(i.min, min(i.max, n - 1)))
                        intervals = intervals.set(rating, Interval(n, i.max))
                    elif op == ">":
                        new_intervals = intervals.set(rating, Interval(max(i.min, n + 1), i.max))
                        intervals = intervals.set(rating, Interval(i.min, n))
                    total += sum_acceptable(res, new_intervals)
        return total
    
    intervals = pmap({c: Interval(1, 4000) for c in "xmas"})
    return sum_acceptable("in", intervals) 

assert part_2(test_input) == 167409079868000

In [11]:
part_2(input)

123972546935551

Unbelievably that code worked first time, which is always a treat.